In [24]:
import pandas as pd
import numpy as np
import pickle as pkl
import datetime as datetime
from tqdm import tqdm
import networkx as nx
import random

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from src.utils.CreateFeatures import CreateFeatures
from src.pygcn.SiameseGNN import SiameseGNN
from src.pygcn.GraphSAGE import SiameseGNN_GraphSAGE
from src.pygcn.graph_isomorphism import SiameseGNN_GIN
from torch_geometric.data import DataLoader
from sklearn.metrics import f1_score, fbeta_score
from src.utils.functions import dist_labels_to_changepoint_labels, dist_labels_to_changepoint_labels_adjusted
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch
import torch_geometric.data as data
from src.synthetic_experiments.sample import sample_pairs
from src.utils.misc import collate
import itertools
from src.pygcn.GAT import SiameseGNN_GAT

## All Events

In [25]:
years = range(1962,2019)

In [26]:
all_nodes = ['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM',
       'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD',
       'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA',
       'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN',
       'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB',
       'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DMA', 'DNK', 'DOM', 'DZA',
       'ECU', 'EGY', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FSM',
       'GAB', 'GBR', 'GEO', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC',
       'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HKG', 'HND', 'HRV', 'HTI',
       'HUN', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA',
       'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KNA', 'KOR',
       'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LKA', 'LSO', 'LTU',
       'LUX', 'LVA', 'MAC', 'MAR', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL',
       'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MNP', 'MOZ', 'MRT',
       'MUS', 'MWI', 'MYS', 'NAM', 'NER', 'NGA', 'NIC', 'NLD', 'NOR',
       'NPL', 'NRU', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW',
       'PNG', 'POL', 'PRT', 'PRY', 'PSE', 'PYF', 'QAT', 'ROU', 'RUS',
       'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLB', 'SLE', 'SLV', 'SMR',
       'SRB', 'SSD', 'STP', 'SUR', 'SVK', 'SVN', 'SWE', 'SWZ', 'SXM',
       'SYC', 'SYR', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TLS', 'TON',
       'TTO', 'TUN', 'TUR', 'TUV', 'TZA', 'UGA', 'UKR', 'URY', 'USA',
       'UZB', 'VCT', 'VEN', 'VNM', 'VUT', 'WSM', 'YEM', 'ZAF', 'ZMB',
       'ZWE']

In [28]:
with open("src/pygcn/graphs_nogdp.pkl", "rb") as f:         
    all_graphs = pkl.load(f)

In [29]:
crisis_years = [1962, 1967, 1973, 1978, 1981, 1989, 1993, 1996, 2002, 2007, 2012, 2014, 2016]
phases = []
p = -1
for i in range(1962,2019):
    if i in crisis_years:
        p += 1
    phases.append(p)

In [ ]:
# labels = dist_labels_to_changepoint_labels(phases)
# graph_pairs = sample_pairs(all_graphs,labels)

In [ ]:
# with open('graph_pairs_nogdp.pkl', 'wb') as f:
#     pkl.dump(graph_pairs, f)

In [ ]:
# with open('graph_pairs.pkl', 'rb') as f:
#     graph_pairs = pkl.load(f)

In [ ]:
# with open('test_data.pkl', 'rb') as f:
#     test_data = pkl.load(f)

# with open('val_data.pkl', 'rb') as f:
#     val_data = pkl.load(f)

# with open('train_data.pkl', 'rb') as f:
#     train_data = pkl.load(f)

In [30]:
with open('test_data_indices_nogdp.pkl', 'rb') as f:
    test_indices = pkl.load(f)

with open('val_data_indices_nogdp.pkl', 'rb') as f:
    val_indices = pkl.load(f)

with open('train_data_indices_nogdp.pkl', 'rb') as f:
    train_indices = pkl.load(f)

In [ ]:
# train_indices, test_indices = train_test_split(np.arange(len(graph_pairs)), test_size=0.40, random_state=42)
# test_indices, val_indices = train_test_split(test_indices, test_size=0.5, random_state=42)

In [ ]:
# graph_pairs_train = [graph_pairs[i] for i in train_indices]
# graph_pairs_test = [graph_pairs[i] for i in test_indices]
# graph_pairs_val = [graph_pairs[i] for i in val_indices]

In [ ]:
# import random
# positive_samples = [item for item in graph_pairs_train if item[2] == 1]
# negative_samples = [item for item in graph_pairs_train if item[2] == 0]

# # Calculate the difference in count
# diff = len(negative_samples) - len(positive_samples)

# # Upsample positive samples
# if diff > 0:
#     positive_samples_upsampled = random.sample(positive_samples, k=abs(diff))
#     balanced_data = negative_samples + positive_samples+ positive_samples_upsampled
# # Downsample negative samples
# elif diff < 0:
#     negative_samples_downsampled = random.sample(negative_samples, k=abs(diff))
#     balanced_data = negative_samples_downsampled + negative_samples+ positive_samples
# else:
#     balanced_data = graph_pairs_train

# random.shuffle(balanced_data)

In [ ]:
# with open('train_data_nogdp.pkl', 'wb') as f:
#     pkl.dump(balanced_data, f)

# with open('val_data_nogdp.pkl', 'wb') as f:
#     pkl.dump(graph_pairs_val, f)
    
# with open('test_data_nogdp.pkl', 'wb') as f:
#     pkl.dump(graph_pairs_test, f)

In [ ]:
# with open('train_data_indices_nogdp.pkl', 'wb') as f:
#     pkl.dump(train_indices, f)

# with open('val_data_indices_nogdp.pkl', 'wb') as f:
#     pkl.dump(val_indices, f)
    
# with open('test_data_indices_nogdp.pkl', 'wb') as f:
#     pkl.dump(test_indices, f)

In [31]:
with open('train_data_nogdp.pkl', 'rb') as f:
    balanced_data = pkl.load(f)

with open('val_data_nogdp.pkl', 'rb') as f:
    graph_pairs_val = pkl.load(f)
    
with open('test_data_nogdp.pkl', 'rb') as f:
    graph_pairs_test = pkl.load(f)

## sGNN with GCN Encoder and 3 Features

In [32]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import f1_score, fbeta_score
import numpy as np

def run_model_cv(model, type, data_pairs, num_folds=5, num_epochs=20):
    
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_scores = []

    val_acc = 0
    val_f1 = 0
    val_f2 = 0
    val_f05 = 0

    for fold, (train_index, val_index) in enumerate(kf.split(data_pairs)):
        print(f"Fold {fold + 1}/{num_folds}")
        if type == 'sage':
            model = SiameseGNN_GraphSAGE(50, 10, 0.1, 16)
        elif type == 'gin':
            model = SiameseGNN_GIN(50, 10, 0.1, 16)
        elif type == 'gat':
            model = SiameseGNN_GAT(50, 10, 0.1, 16)
        else:
            model = SiameseGNN(50, 10, 0.1, 16)
        torch.manual_seed(42)
        optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.0001)
        scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
        criterion = nn.BCELoss()  # Changed to BCEWithLogitsLoss for numerical stability

        # Prepare data for current fold
        training_data_pairs = [data_pairs[i] for i in train_index]
        validation_data_pairs = [data_pairs[i] for i in val_index]

        for epoch in tqdm(range(num_epochs)):
            model.train()
            train_losses = []
            for (graph1, graph2, labels) in training_data_pairs:
                optimizer.zero_grad()
                out = model(graph1, graph2)

                loss = criterion(out, labels)
                loss.backward()
                optimizer.step()
                train_losses.append(loss.item())

            scheduler.step()

            model.eval()
            with torch.no_grad():
                val_losses = []
                val_pred = []
                val_truth = []

                correct = 0
                total = 0
                for (graph1, graph2, labels) in validation_data_pairs:
                    out = model(graph1, graph2)

                    val_loss = criterion(out, labels)
                    val_losses.append(val_loss.item())

                    predictions = torch.round(out)

                    val_pred.extend(predictions.cpu().numpy())
                    val_truth.extend(labels.cpu().numpy())

                    correct += (predictions == labels).sum().item()
                    total += labels.size(0)

                val_loss = sum(val_losses) / len(val_losses)
                val_accuracy = correct / total

            val_f1 = f1_score(val_truth, val_pred)
            val_f2 = fbeta_score(y_true=val_truth, y_pred=val_pred, beta=2)
            val_f05 = fbeta_score(y_true=val_truth, y_pred=val_pred, beta=1 / 2)

            print(f'Fold: {fold + 1}, Epoch: {epoch + 1}, Training Loss: {sum(train_losses) / len(train_losses)}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}, Validation F1 Score: {val_f1}, Validation F2 Score: {val_f2}, Validation F0.5 Score: {val_f05}')

            if epoch ==19:
                val_acc += val_accuracy
                val_f1 += val_f1
                val_f2 += val_f2
                val_f05 += val_f05

        model_name = f"models/real_data/sage-fold{fold}.pt"
        torch.save(model.state_dict(), model_name)

    fold_scores.append((val_acc / 5, val_f1 / 5, val_f2 / 5, val_f05 / 5))

    return fold_scores

In [33]:
train_data = balanced_data + graph_pairs_val

In [35]:
input_dim = train_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "gcn", train_data)

Fold 1/5


  5%|▌         | 1/20 [00:15<04:59, 15.78s/it]

Fold: 1, Epoch: 1, Training Loss: 0.6909637743122525, Validation Loss: 0.7123753621242941, Validation Accuracy: 0.50625, Validation F1 Score: 0.6721991701244814, Validation F2 Score: 0.8367768595041323, Validation F0.5 Score: 0.5617198335644937


 10%|█         | 2/20 [00:30<04:37, 15.44s/it]

Fold: 1, Epoch: 2, Training Loss: 0.6913682036218598, Validation Loss: 0.7089795511215925, Validation Accuracy: 0.50625, Validation F1 Score: 0.6721991701244814, Validation F2 Score: 0.8367768595041323, Validation F0.5 Score: 0.5617198335644937


...

Fold: 5, Epoch: 18, Training Loss: 0.6907781040005695, Validation Loss: 0.6901662187142805, Validation Accuracy: 0.5297805642633229, Validation F1 Score: 0.6926229508196722, Validation F2 Score: 0.8492462311557789, Validation F0.5 Score: 0.5847750865051903


 95%|█████████▌| 19/20 [05:03<00:17, 17.19s/it]

Fold: 5, Epoch: 19, Training Loss: 0.6895989090460954, Validation Loss: 0.6897865898183146, Validation Accuracy: 0.5297805642633229, Validation F1 Score: 0.6926229508196722, Validation F2 Score: 0.8492462311557789, Validation F0.5 Score: 0.5847750865051903


100%|██████████| 20/20 [05:19<00:00, 15.98s/it]

Fold: 5, Epoch: 20, Training Loss: 0.6893918666413918, Validation Loss: 0.6893667713228064, Validation Accuracy: 0.5297805642633229, Validation F1 Score: 0.6926229508196722, Validation F2 Score: 0.8492462311557789, Validation F0.5 Score: 0.5847750865051903


In [ ]:
input_dim = balanced_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GraphSAGE(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "sage", train_data)

In [ ]:
input_dim = balanced_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GIN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "gin", train_data)

In [ ]:
input_dim = train_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GAT(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "gat", train_data)

fold_scores

## sGNN with Feature Subset

In [ ]:
with open("feature_dicts/mis_features_not_norm.pkl", "rb") as f:
    feat_dict = pkl.load(f)

In [ ]:
def add_features(years, graphs, feat_dict, dim):

    zeros = torch.zeros(dim)

    for i in range(len(years)):
        new_x = torch.empty(0, dim)
        year = years[i]

        feat_dict_year = feat_dict[year].combined_features

        for j, country in enumerate(all_nodes):
            if j == 0:
                new_x = torch.stack([zeros])

            elif country in feat_dict_year["country_code"].values:
                tensor_before = graphs[i].x[j]
                country_row = feat_dict_year[feat_dict_year["country_code"] == country]
                country_row = country_row.drop(columns = ["country_code", "current_gdp_growth"])
                row_values = country_row.values.tolist()
                row_tensor = torch.tensor(row_values)[0]
                combined_values = torch.cat((tensor_before, row_tensor))

                new_x = torch.cat((new_x, combined_values.unsqueeze(0)), dim=0)

            else:
                new_x = torch.cat((new_x, zeros.unsqueeze(0)), dim=0)

        graphs[i].x = new_x

    return graphs

In [ ]:
with open("src/pygcn/all_graphs.pkl", "rb") as f:         
    all_graphs = pkl.load(f)

In [ ]:
feat_dict[1962].combined_features

In [ ]:
all_graphs = add_features(years, all_graphs, feat_dict, 27)

In [ ]:
labels = dist_labels_to_changepoint_labels(phases)
graph_pairs = sample_pairs(all_graphs,labels)

In [ ]:
graph_pairs_train = [graph_pairs[i] for i in train_indices]
graph_pairs_test = [graph_pairs[i] for i in test_indices]
graph_pairs_val = [graph_pairs[i] for i in val_indices]

In [ ]:
import random
positive_samples = [item for item in graph_pairs_train if item[2] == 1]
negative_samples = [item for item in graph_pairs_train if item[2] == 0]

# Calculate the difference in count
diff = len(negative_samples) - len(positive_samples)

# Upsample positive samples
if diff > 0:
    positive_samples_upsampled = random.sample(positive_samples, k=abs(diff))
    balanced_data = negative_samples + positive_samples+ positive_samples_upsampled
# Downsample negative samples
elif diff < 0:
    negative_samples_downsampled = random.sample(negative_samples, k=len(negative_samples))
    balanced_data = negative_samples_downsampled + negative_samples+ positive_samples
else:
    balanced_data = graph_pairs_train

random.shuffle(balanced_data)

In [ ]:
train_data = balanced_data + graph_pairs_val

In [ ]:
input_dim = train_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "gcn", train_data)

In [ ]:
input_dim = train_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GraphSAGE(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "sage", train_data)

In [ ]:
input_dim = train_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GIN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "gin", train_data)

In [ ]:
input_dim = train_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GAT(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "gat", train_data)

## Random Feature Subset

In [ ]:
years = range(1962,2019)

In [ ]:
with open("feature_dicts/random_dict_not_norm.pkl", "rb") as f:
    feat_dict_random = pkl.load(f)

In [ ]:
with open("src/pygcn/all_graphs.pkl", "rb") as f:         
    all_graphs = pkl.load(f)

In [ ]:
all_graphs = add_features(years, all_graphs, feat_dict_random, 27)

In [ ]:
labels = dist_labels_to_changepoint_labels(phases)
graph_pairs = sample_pairs(all_graphs,labels)

In [ ]:
graph_pairs_train = [graph_pairs[i] for i in train_indices]
graph_pairs_test = [graph_pairs[i] for i in test_indices]
graph_pairs_val = [graph_pairs[i] for i in val_indices]

In [ ]:
import random
positive_samples = [item for item in graph_pairs_train if item[2] == 1]
negative_samples = [item for item in graph_pairs_train if item[2] == 0]

# Calculate the difference in count
diff = len(negative_samples) - len(positive_samples)

# Upsample positive samples
if diff > 0:
    positive_samples_upsampled = random.sample(positive_samples, k=abs(diff))
    balanced_data = negative_samples + positive_samples+ positive_samples_upsampled
# Downsample negative samples
elif diff < 0:
    negative_samples_downsampled = random.sample(negative_samples, k=len(negative_samples))
    balanced_data = negative_samples_downsampled + negative_samples+ positive_samples
else:
    balanced_data = graph_pairs_train

random.shuffle(balanced_data)

In [ ]:
train_data = balanced_data + graph_pairs_val

In [ ]:
input_dim = train_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "gcn", train_data)

In [ ]:
input_dim = train_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GraphSAGE(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "sage", train_data)

In [ ]:
input_dim = train_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GAT(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "gin", train_data)

In [ ]:
input_dim = train_data[0][0].x.shape[1]

# Define hyperparameter grids
learning_rates = [1e-4]
dropout_rates = [0.1]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GIN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    fold_scores = run_model_cv(model, "gat", train_data)

## Log Transformed Features